# Title

In [3]:
"""
    TITLE   :
    PROJECT :
""";

__author__ = ''
__version__ = ''

<span style='font-size:30px;font-weight:650'>
    About
</span>

information about this notebook


<br><br>

- - - 

## Prepare


### Imports

In [4]:
# THIRD PARTY
import numpy as np

import astropy.coordinates as coord
import astropy.units as u
from astropy.visualization import quantity_support

from galpy import potential as gpot
from galpy import df as gdf
import agama

# PROJECT-SPECIFIC
from discO import conf
from discO import PotentialSampler, GaussianMeasurementErrorSampler
from discO.extern.galpy import GalpyPotentialSampler
from discO.extern.agama import AGAMAPotentialSampler

In [6]:
quantity_support();

### Parameters

In [7]:
mass = 1e12 * u.solMass
r0 = 10 * u.kpc  # scale factor

<br><br>

- - - 

## Code

In [10]:
hernquist_pot = gpot.HernquistPotential(amp=mass, a=r0)

In [11]:
sampler = PotentialSampler(gdf.isotropicHernquistdf(hernquist_pot))
sampler.sample(n=10)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [( 57.28967844, -10.24552944,  15.59416962),
     ( 35.18365664, -13.87658697,  71.11778097),
     (151.50179475,  25.44896814,  27.73565783),
     ( 29.31768639,  39.76503984, 277.36005243),
     (290.29370181, -35.96801609,  21.65758233),
     (251.27700789,   3.59458879,  39.71698392),
     (319.80717188, -44.36936295,  14.05643205),
     (335.65907837,  42.42438802,  49.67683221),
     (336.50197028,  14.46312243,  24.20469522),
     (298.10838653, -23.07323642, 216.83722675)]
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    [( 3.7712906 , -1.77875047,  -29.91516669),
     ( 0.86418893, -0.87769723,   58.37524997),
     (-0.45555229, -2.07863305,  122.06665169),
     ( 0.10468691, -0.07270632, -109.16216059),
     ( 0.41971767, -1.74725582,  -10.65030709),
     (-0.86696437, -0.68452346, -179.21575502),
     ( 0.59928085, -2.79722695,  -22.42080189),
     ( 0.16589981,  0.25276936, -312.3714720

In [32]:
array = sampler.resample(niter=20, n=3)

array[0, 1]

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [(253.32843542, -38.64375696, 70.9631958 ),
     (278.24396417,  29.24388061,  5.64393122),
     (344.2517297 , -49.68718139, 44.69146923)]
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    [(-0.41850885, -0.34364907,   14.18413179),
     ( 5.04910284, -1.72523734, -175.77680393),
     ( 0.89364931, -0.75713715,    3.38484785)]>

In [33]:
sampler._instance

AGAMA

In [27]:
pot_Hern = agama.Potential(type='Spheroid', mass=1e12, scaleRadius=10, gamma=1, alpha =1, beta=4, cutoffStrength=0)

In [36]:
sampler = PotentialSampler(
    agama.Potential(type='Spheroid', mass=1e12, scaleRadius=10, gamma=1, alpha =1, beta=4, cutoffStrength=0),
)

sampler

In [37]:
samples = sampler.sample(n=10)
samples

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [(214.46359204,  46.31018846,   7.75971433),
     (203.44129085,  72.46682219,   1.958076  ),
     (228.88249676, -58.99884576,   4.73988409),
     (112.68609327, -53.87903217,  19.2007379 ),
     ( 35.52853245,  -7.63588012,  11.72565254),
     (114.5947318 ,  75.93097341,  99.55560549),
     (240.5978794 ,   6.54805444, 823.63128559),
     ( 98.7684863 ,  30.60378122,  20.77447887),
     (223.97787116, -72.92522127,   1.55342859),
     (305.10078681, -60.88181598,  43.09081437)]>

In [38]:
samples.potential

In [41]:
array = sampler.resample(niter=20, n=3)

array

array([[<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [( 85.82871899,  29.43431962, 80.19898726),
     (322.82202395,  -2.02430613, 57.42167078),
     (316.90798595, -51.2351452 , 10.75122914)]>,
        <SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [(112.16397588, -15.62163995, 198.73845272),
     (215.35836886,  58.21797693,  50.79114918),
     (349.31128296,  15.89803886,  14.10723674)]>,
        <SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [( 33.25883985,  56.52342483, 22.08826193),
     (270.36090699, -37.91184112, 96.31478025),
     (350.21451467, -27.43959089, 20.37785779)]>,
        <SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [(135.38108794, -8.63218079, 75.5545025 ),
     ( 44.15306276, 26.26362322, 16.39688817),
     (188.38280938, 56.31265872, 11.20043373)]>,
        <SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, kpc)
    [(  3.38475812,  -5.22624916, 10.82405519),
     (179.63905266,  71.07575689, 58.146771

---

Now pipe with fitters

In [ ]:
sampler = PotentialSampler(gdf.isotropicHernquistdf(hernquist_pot))
fitter = PotentialFitter("multipole", package="agama", return_specific_class=True)
ms = GaussianMeasurementErrorSampler()

pipe = sampler | ms | fitter

fit_pot = pipe(n=10000)

fit_pot

In [ ]:
ms(sampler.sample(n=3), c_err=0.01)

<br><br>

- - - 

<span style='font-size:40px;font-weight:650'>
    END
</span>